In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# Conexión a un archivo CSV preprocesado perteneciente a la DB MIMIC-III
# Lectura del archivo CSV en un DataFrame de Pandas
df = pd.read_csv(r'C:\Users\rocio\OneDrive\Escritorio\henry\M6\Clase 01\mimic\modelo.csv')

In [3]:
df.head()

,subject_id,fecha,Arterial BP [Systolic],Arterial Blood Pressure mean,Arterial Blood Pressure systolic,Bicarbonate,"Bilirubin, Total",Creatinine,GCS Total,Heart Rate,Platelet Count,"Potassium, Whole Blood",Respiratory Rate,"Sodium, Whole Blood",Temperature Fahrenheit,Urea Nitrogen,Vancomycin/Random,WBC Count,pO2
0,10006,2164-09-24,NaN,NaN,NaN,27.0,NaN,7.0,NaN,NaN,157.0,NaN,NaN,NaN,NaN,31.0,NaN,NaN,NaN
1,10006,2164-09-25,NaN,NaN,NaN,28.0,NaN,7.4,NaN,NaN,168.0,NaN,NaN,NaN,NaN,34.0,NaN,NaN,NaN
2,10006,2164-09-26,NaN,NaN,NaN,28.0,NaN,8.3,NaN,NaN,152.0,NaN,NaN,NaN,NaN,42.0,NaN,NaN,NaN
3,10006,2164-09-27,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,127.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,10006,2164-09-28,NaN,NaN,NaN,27.0,NaN,6.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,28.0,NaN,NaN,NaN


In [5]:
# Renombra las columnas del DataFrame
df = df.rename(columns={
    'subject_id': 'paciente',
    'fecha': 'fecha',
    'Arterial BP [Systolic]': 'PAS',
    'Arterial Blood Pressure mean': 'PAM',
    'Arterial Blood Pressure systolic': 'sistolica',
    'Bicarbonate': 'bicarbonato',
    'Bilirubin, Total': 'bilirrubina',
    'Creatinine': 'creatinina',
    'GCS Total': 'GCS',
    'Heart Rate': 'ritmo_cardiaco',
    'Platelet Count': 'plaquetas',
    'Potassium, Whole Blood': 'potasio',
    'Whole Blood': 'sangre_total',
    'Respiratory Rate': 'ritmo_respiratorio',
    'Sodium, Whole Blood': 'sodio',
    'Temperature Fahrenheit': 'temperatura_F',
    'Urea Nitrogen': 'urea',
    'Vancomycin/Random': 'vancomicina',
    'WBC Count': 'glucemia',
    'pO2': 'pO2'
})


In [6]:
#Visualizo
df.head()

,paciente,fecha,PAS,PAM,sistolica,bicarbonato,bilirrubina,creatinina,GCS,ritmo_cardiaco,plaquetas,potasio,ritmo_respiratorio,sodio,temperatura_F,urea,vancomicina,glucemia,pO2
0,10006,2164-09-24,NaN,NaN,NaN,27.0,NaN,7.0,NaN,NaN,157.0,NaN,NaN,NaN,NaN,31.0,NaN,NaN,NaN
1,10006,2164-09-25,NaN,NaN,NaN,28.0,NaN,7.4,NaN,NaN,168.0,NaN,NaN,NaN,NaN,34.0,NaN,NaN,NaN
2,10006,2164-09-26,NaN,NaN,NaN,28.0,NaN,8.3,NaN,NaN,152.0,NaN,NaN,NaN,NaN,42.0,NaN,NaN,NaN
3,10006,2164-09-27,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,127.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,10006,2164-09-28,NaN,NaN,NaN,27.0,NaN,6.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,28.0,NaN,NaN,NaN


In [7]:
#Verifico si todos los registros son unicos
count = df.nunique()
print(count)

paciente               100
fecha                 1649
PAS                     80
PAM                     58
sistolica               73
bicarbonato             40
bilirrubina            116
creatinina              92
GCS                     14
ritmo_cardiaco          84
plaquetas              456
potasio                 35
ritmo_respiratorio      36
sodio                   18
temperatura_F           68
urea                   135
vancomicina             47
glucemia                 1
pO2                    182
dtype: int64


In [8]:
# Cuenta la cantidad de valores nulos en cada fila del DataFrame
null_counts = df.isnull().sum(axis=1)

# Imprime los resultados
print(null_counts)


0       13
1       13
2       13
3       16
4       14
        ..
1713    12
1714    13
1715    13
1716    13
1717    16
Length: 1718, dtype: int64


In [ ]:
#Elimino aquellos registros cuyos nulos sean >60 % (ya que si tiene tantos valores faltantes no me va a servir para entrenar el modelo)

# Define el umbral de valores no nulos
threshold = len(df.columns) * 0.4

# Elimina las filas que no cumplen el umbral de valores no nulos
df = df.dropna(thresh=threshold)


In [ ]:
#Corroboro
df.head()

In [ ]:
# Cuenta la cantidad de valores nulos en cada fila del DataFrame
null_counts = df.isnull().sum(axis=1)

# Imprime los resultados
print(null_counts)

In [ ]:
#Selecciono pacientes que hayan tenido como mínimo 3 mediciones de signos vitales

# Agrupar por paciente y contar registros no nulos por columna de signos vitales
counts = df.groupby('paciente').agg({'PAS': 'count',
                                       'PAM': 'count',
                                       'bicarbonato': 'count',
                                       'bilirrubina': 'count',
                                       'creatinina': 'count',
                                       'GCS': 'count',
                                       'ritmo_cardiaco': 'count',
                                       'plaquetas': 'count',
                                       'potasio': 'count',
                                       'ritmo_respiratorio': 'count',
                                       'sodio': 'count',
                                       'temperatura_F': 'count',
                                       'urea': 'count',
                                       'glucemia': 'count',
                                       'pO2': 'count'})

# Seleccionar solamente los registros de pacientes con al menos 3 registros de signos vitales
selected_subject_ids = counts[counts['PAS'] >= 3].index

# Filtrar los datos del DataFrame original por los pacientes seleccionados
df_selected = df[df['paciente'].isin(selected_subject_ids)]


In [10]:
#Reemplazo los nulos y NaN que quedaron por la media de cada columna
df.fillna(df.mean(), inplace=True)
df.fillna(value=df.mean(), inplace=True)


C:\Users\rocio\AppData\Local\Temp\ipykernel_27984\393412877.py:2: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  df.fillna(df.mean(), inplace=True)
C:\Users\rocio\AppData\Local\Temp\ipykernel_27984\393412877.py:3: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  df.fillna(value=df.mean(), inplace=True)


In [12]:
# Cuenta la cantidad de valores nulos en cada fila del DataFrame
null_counts = df.isnull().sum(axis=1)

# Imprime los resultados
print(null_counts)


0       0
1       0
2       0
3       0
4       0
       ..
1713    0
1714    0
1715    0
1716    0
1717    0
Length: 1718, dtype: int64


In [14]:
df.head()

,paciente,fecha,PAS,PAM,sistolica,bicarbonato,bilirrubina,creatinina,GCS,ritmo_cardiaco,plaquetas,potasio,ritmo_respiratorio,sodio,temperatura_F,urea,vancomicina,glucemia,pO2
0,10006,2164-09-24,180.994975,76.453237,119.702899,27.000000,5.5329,7.000000,23.981651,92.660606,157.000000,4.053191,19.77439,137.539683,98.078841,31.000000,93.003509,3.1,124.88191
1,10006,2164-09-25,180.994975,76.453237,119.702899,28.000000,5.5329,7.400000,23.981651,92.660606,168.000000,4.053191,19.77439,137.539683,98.078841,34.000000,93.003509,3.1,124.88191
2,10006,2164-09-26,180.994975,76.453237,119.702899,28.000000,5.5329,8.300000,23.981651,92.660606,152.000000,4.053191,19.77439,137.539683,98.078841,42.000000,93.003509,3.1,124.88191
3,10006,2164-09-27,180.994975,76.453237,119.702899,25.832915,5.5329,1.617455,23.981651,92.660606,127.000000,4.053191,19.77439,137.539683,98.078841,31.360939,93.003509,3.1,124.88191
4,10006,2164-09-28,180.994975,76.453237,119.702899,27.000000,5.5329,6.200000,23.981651,92.660606,212.078851,4.053191,19.77439,137.539683,98.078841,28.000000,93.003509,3.1,124.88191


In [16]:
df.to_csv('df_selected.csv', index=False)

Entrenamiento de modelo de machine learning


In [24]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split


# Contar los valores de "PAS" para cada paciente
counts = df.groupby('paciente')['PAS'].count()

# Seleccionar solo los pacientes que tienen al menos 3 valores de "PAS"
selected_subject_ids = counts[counts >= 3].index
df = df[df['paciente'].isin(selected_subject_ids)]

# Definir la variable objetivo 'sepsis'
df['sepsis'] = ((df['ritmo_cardiaco'] >= 22) & 
                         (df['GCS'] < 15) &
                         (df['PAS'] <= 100)).astype(int)

# Seleccionar los signos vitales relevantes
X = df[['ritmo_cardiaco', 'GCS', 'PAS']]
y = df['sepsis']

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Crear y entrenar el modelo de árbol de decisión
model = DecisionTreeClassifier(random_state=42)
model.fit(X_train, y_train)

# Evaluar el modelo
train_score = model.score(X_train, y_train)
test_score = model.score(X_test, y_test)
print(f'Train score: {train_score:.3f}, Test score: {test_score:.3f}')




Train score: 1.000, Test score: 1.000


In [25]:
# Crear un conjunto de características con los signos vitales del paciente
paciente = [[50, 50, 200]]

# Obtener la predicción del modelo
prediccion = model.predict(paciente)

# Imprimir la predicción
print("El paciente tiene sepsis" if prediccion == 1 else "El paciente no tiene sepsis")


El paciente no tiene sepsis


c:\Users\rocio\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(
